<a href="https://colab.research.google.com/github/POOJACH76/API_ecommerce/blob/master/Malaria.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
! mkdir -p ~/.kaggle

! cp kaggle.json ~/.kaggle/

In [7]:
!kaggle datasets download -d iarunava/cell-images-for-detecting-malaria

 97% 658M/675M [00:07<00:00, 18.2MB/s]
100% 675M/675M [00:08<00:00, 88.5MB/s]


In [8]:
import zipfile

In [9]:
zip_ref= zipfile.ZipFile("/content/cell-images-for-detecting-malaria.zip",'r')
zip_ref.extractall('/content')

zip_ref.close()

In [10]:
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
import warnings
warnings.filterwarnings("ignore")

In [11]:
print(tf.__version__)

2.15.0


In [12]:
IMAGE_SIZE=[224,224]
train_path="/content/cell_images/Parasitized"
validation_path="/content/cell_images/Uninfected"

In [13]:

datagen = ImageDataGenerator(featurewise_center=False,
                             samplewise_center=False,
                             featurewise_std_normalization=False,
                             samplewise_std_normalization=False,
                             zca_whitening=False,
                             rotation_range=10,
                             zoom_range=0.1,
                             width_shift_range=0.1,
                             height_shift_range=0.1,
                             horizontal_flip=False,
                             vertical_flip=False,
                             rescale=1/255.0,
                             validation_split=0.2)

In [14]:
from keras.applications.xception import Xception, preprocess_input
xception_wo_top = Xception(include_top=False, weights='imagenet', input_tensor=None, input_shape=(71,71,3), pooling=None)

83683744/83683744 [==============================] - 0s 0us/step


In [15]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(rescale=1.0/255)

Data = datagen.flow_from_directory(
    directory='/content/cell_images/cell_images',
    target_size=(71, 71),
    class_mode='binary',
    batch_size=16,
    subset='training',
)



Found 27558 images belonging to 2 classes.


In [16]:
#Normalize
def process (image,label):
  image= tf.cast(image/255. ,tf.float32)
  return image,label
  acne_ds= acne_ds.map(process)

In [17]:
model = Sequential()

model.add(Conv2D(32, kernel_size=(3, 3), padding='valid', activation='relu', input_shape=(256, 256, 3)))
model.add(MaxPooling2D(pool_size=(2, 2), strides=2, padding='valid'))

model.add(Conv2D(64, kernel_size=(3, 3), padding='valid', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=2, padding='valid'))

model.add(Conv2D(128, kernel_size=(3, 3), padding='valid', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=2, padding='valid'))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 254, 254, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 127, 127, 32)      0         
 D)                                                              
                                                                 
 conv2d_5 (Conv2D)           (None, 125, 125, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 62, 62, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_6 (Conv2D)           (None, 60, 60, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 30, 30, 128)       0

In [18]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import accuracy_score, precision_score, recall_score

datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    validation_split=0.2,
    rotation_range=10,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=False,
    vertical_flip=False
)

train_datagen = datagen.flow_from_directory(
    directory='/content/cell_images',
    target_size=(71, 71),
    class_mode='binary',
    batch_size=16,
    subset='training'
)

valid_datagen = datagen.flow_from_directory(
    directory='/content/cell_images',
    target_size=(71, 71),
    class_mode='binary',
    batch_size=16,
    subset='validation'
)


model = Sequential()

model.add(Conv2D(32, kernel_size=(3, 3), padding='valid', activation='relu', input_shape=(71, 71, 3)))
model.add(MaxPooling2D(pool_size=(2, 2), strides=2, padding='valid'))
model.add(Conv2D(64, kernel_size=(3, 3), padding='valid', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=2, padding='valid'))
model.add(Conv2D(128, kernel_size=(3, 3), padding='valid', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=2, padding='valid'))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])


num_epochs = 10
model.fit(train_datagen, epochs=num_epochs, validation_data=valid_datagen)

test_datagen = datagen.flow_from_directory(
    directory='/content/cell_images',
    target_size=(71, 71),
    class_mode='binary',
    batch_size=16,
    subset='validation',
    shuffle=False
)

predictions = model.predict(test_datagen)
predicted_labels = (predictions > 0.5).astype(int)
true_labels = test_datagen.labels

# Calculate metrics
accuracy = accuracy_score(true_labels, predicted_labels)
precision = precision_score(true_labels, predicted_labels, average='micro')
recall = recall_score(true_labels, predicted_labels, average='micro')


print(f'Accuracy: {accuracy:.2f}')
print(f'Precision: {precision:.2f}')
print(f'Recall: {recall:.2f}')


Found 44095 images belonging to 3 classes.
Found 11021 images belonging to 3 classes.
Epoch 1/10
2756/2756 [==============================] - 173s 61ms/step - loss: -101951492587520.0000 - accuracy: 0.2500 - val_loss: -516987662893056.0000 - val_accuracy: 0.2500
Epoch 2/10
2756/2756 [==============================] - 160s 58ms/step - loss: -3198883320561664.0000 - accuracy: 0.2500 - val_loss: -7822700424724480.0000 - val_accuracy: 0.2500
Epoch 3/10
2756/2756 [==============================] - 160s 58ms/step - loss: -20199439499001856.0000 - accuracy: 0.2500 - val_loss: -36580880705126400.0000 - val_accuracy: 0.2500
Epoch 4/10
2756/2756 [==============================] - 156s 57ms/step - loss: -71446742314254336.0000 - accuracy: 0.2500 - val_loss: -111999698969034752.0000 - val_accuracy: 0.2500
Epoch 5/10
2756/2756 [==============================] - 169s 61ms/step - loss: -187016499704102912.0000 - accuracy: 0.2500 - val_loss: -267447304284274688.0000 - val_accuracy: 0.2500
Epoch 6/10
2

In [19]:
# Save the trained model using TensorFlow's built-in method
model.save("trained_model.h5")

# Later, you can load the model using TensorFlow
loaded_model = tf.keras.models.load_model("trained_model.h5")

In [20]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model


model = load_model('trained_model.h5')


test_img = cv2.imread('/content/maleria.png')
test_img = cv2.resize(test_img, (71, 71))
test_img = test_img / 255.0
test_input = np.expand_dims(test_img, axis=0)


predictions = model.predict(test_input)


class_labels = ["infected","Not infected"]


predicted_class_index = np.argmax(predictions)
predicted_class = class_labels[predicted_class_index]

plt.imshow(test_img)
plt.title(f"Prediction: {predicted_class}")
plt.show()


error: OpenCV(4.8.0) /io/opencv/modules/imgproc/src/resize.cpp:4062: error: (-215:Assertion failed) !ssize.empty() in function 'resize'
